In [2]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
import pandas as pd
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard

In [3]:
df = pd.read_csv ('/Users/abby/Documents/fa21-neural-networks/final_project/all-the-news-2-1.csv')

/Users/abby/Applications/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')

In [9]:
df_fewer_cols = df[['article','title','publication']]

In [15]:
df_vox = df_fewer_cols[df_fewer_cols['publication'] == 'Vox'].article

In [20]:
df_vox

0          This post is part of Polyarchy, an independent...
8          Google I/O, the company's big developer confer...
10         Elizabeth Warren is giving people a new reason...
15         Nancy Pelosi is getting visions of reclaiming ...
16         The nonpartisan director of the federal Office...
                                 ...                        
2674053    The United States continues to record far and ...
2674054    Zoom, the videoconferencing app that’s dominat...
2674055    Governors in several of the hardest-hit states...
2674056    New York City’s Commission on Human Rights wil...
2674057    On Tuesday, the White House’s coronavirus task...
Name: article, Length: 47272, dtype: object

In [19]:
df_vox[0]

'This post is part of Polyarchy, an independent blog produced by the political reform program at New America, a Washington think tank devoted to developing new ideas and new voices. Imagine you are an otherwise healthy 30-something who starts feeling weird. You are sometimes short of breath. You get migraines. Your feet start to swell a little. But otherwise, everything seems fine. You go to the doctor. The doctor runs some tests. She tells you,\xa0It\'s probably nothing, but these could be signs of a coming heart attack. You push for more certainty, but the doctor tells you she\'s not sure. The human body is a complex system. You\'re young and otherwise pretty healthy. There could be plenty of other explanations for what you\'re feeling. But it is a little worrying. So just to be on the safe side, maybe you should reduce the stress in your life and eat a healthier diet. What would you do? If you\'re a sensible person, you\'d probably err on the side of precaution. Sure, it might be no

In [23]:
df.groupby('publication').count()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section
publication,,,,,,,,,,,
Axios,47815,47815,47815,47815,47815,47815,47806,47815,47311,47815,0
Business Insider,57953,57953,57953,57953,57953,57953,54291,57953,57934,57953,0
Buzzfeed News,32819,32819,32819,32819,32819,32819,32819,32819,32724,32819,0
CNBC,238096,238096,238096,238096,238096,238096,1918,238093,234141,238096,193560
CNN,127602,127602,127602,127602,127602,127602,123337,127600,125300,127602,127602
Economist,26227,26227,26227,26227,26227,26227,0,26227,23200,26227,26227
Fox News,20144,20144,20144,20144,20144,20144,20144,20144,20144,20144,20144
Gizmodo,27228,27228,27228,27228,27228,27228,27228,27228,27226,27228,18214
Hyperallergic,13551,13551,13551,13551,13551,13551,11305,13551,13539,13551,0


In [22]:
df[df['publication']=='Vox'].groupby('year').count()

,Unnamed: 0,Unnamed: 0.1,date,month,day,author,title,article,url,section,publication
year,,,,,,,,,,,
2016,13206,13206,13206,13206,13206,13205,13206,13200,13206,0,13206
2017,12145,12145,12145,12145,12145,12142,12145,12145,12145,0,12145
2018,11072,11072,11072,11072,11072,11011,11072,11072,11072,0,11072
2019,8715,8715,8715,8715,8715,8712,8715,8715,8715,0,8715
2020,2134,2134,2134,2134,2134,2134,2134,2133,2134,0,2134
